In [9]:
import pandas as pd
import re
import requests
import certifi
from bs4 import BeautifulSoup

In [1]:
import requests
import pandas as pd
from datetime import datetime

def fetch_ecb_data(url, start_period='2000-01', end_period=None):
    if end_period is None:
        end_period = datetime.now().strftime('%Y-%m')

    params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': 'dataonly'
    }
    
    response = requests.get(url, params=params, headers={'Accept': 'application/json'})
    
    if response.status_code != 200:
        print(f"Error fetching data: {response.status_code}")
        return None

    return response.json()

def parse_data(data, series_key):
    observations = data['dataSets'][0]['series'][series_key]['observations']
    dates = data['structure']['dimensions']['observation'][0]['values']
    records = []
    for key, value in observations.items():
        date_idx = key.split(':')[0]
        date_str = dates[int(date_idx)]['id']
        date = datetime.strptime(date_str, '%Y-%m-%d' if len(date_str) == 10 else '%Y-%m')
        records.append((date, value[0]))
    return pd.DataFrame(records, columns=['Date', series_key])

# URLs for the different data series
url_interest_rate = 'https://data-api.ecb.europa.eu/service/data/FM/M.U2.EUR.4F.KR.MRR_FR.LEV'
url_hicp = 'https://data-api.ecb.europa.eu/service/data/ICP/M.U2.N.000000.4.ANR'
url_unemployment = 'https://data-api.ecb.europa.eu/service/data/LFSI/M.U2.S.UNEH.RTT000.4.A'

# Fetch data
interest_rate_data = fetch_ecb_data(url_interest_rate)
hicp_data = fetch_ecb_data(url_hicp)
unemployment_data = fetch_ecb_data(url_unemployment)

# Parse data
df_interest_rate = parse_data(interest_rate_data, 'Interest Rate')
df_hicp = parse_data(hicp_data, 'HICP')
df_unemployment = parse_data(unemployment_data, 'Unemployment Rate')

# Merge data into a single DataFrame
df_combined = pd.merge(df_interest_rate, df_hicp, on='Date', how='outer')
df_combined = pd.merge(df_combined, df_unemployment, on='Date', how='outer')

# Sort by date
df_combined = df_combined.sort_values(by='Date')

# Save to CSV
df_combined.to_csv('combined_data.csv', index=False)

print("Combined data has been saved to CSV file.")


Error fetching data: 404
Error fetching data: 404


TypeError: 'NoneType' object is not subscriptable

In [41]:
mro=pd.read_csv('https://data-api.ecb.europa.eu/service/data/FM/D.U2.EUR.4F.KR.MRR_FR.LEV?startPeriod=2000-01&detail=dataonly&format=csvdata')
mro=mro[['TIME_PERIOD','OBS_VALUE']]
mro.TIME_PERIOD=pd.to_datetime(mro.TIME_PERIOD, format='%Y-%m-%d')
mro.set_index('TIME_PERIOD',inplace=True)
mro.resample('M').last()

,OBS_VALUE
TIME_PERIOD,
2000-01-31,3.00
2000-02-29,3.25
2000-03-31,3.50
2000-04-30,3.75
2000-05-31,3.75
...,...
2024-01-31,4.50
2024-02-29,4.50
2024-03-31,4.50


In [68]:
mro=pd.read_csv('https://data-api.ecb.europa.eu/service/data/FM/D.U2.EUR.4F.KR.MRR_FR.LEV?startPeriod=2000-01&detail=dataonly&format=csvdata')
mro=mro[['TIME_PERIOD','OBS_VALUE']]
mro.TIME_PERIOD=pd.to_datetime(mro.TIME_PERIOD, format='%Y-%m-%d')
mro.set_index('TIME_PERIOD',inplace=True)
mro=mro.resample('M').last()
mro=mro.rename(columns={'OBS_VALUE':'MRO'})
mro.index=mro.index.strftime('%b-%Y')


inf=pd.read_csv('https://data-api.ecb.europa.eu/service/data/ICP/M.U2.N.000000.4.ANR?startPeriod=2000-01&detail=dataonly&format=csvdata')
inf=inf[['TIME_PERIOD','OBS_VALUE']]
inf.TIME_PERIOD=pd.to_datetime(inf.TIME_PERIOD, format='%Y-%m')
inf.set_index('TIME_PERIOD',inplace=True)
inf=inf.rename(columns={'OBS_VALUE':'Inflacion'})
inf.index=inf.index.strftime('%b-%Y')


une=pd.read_csv('https://data-api.ecb.europa.eu/service/data/LFSI/M.I9.S.UNEHRT.TOTAL0.15_74.T?startPeriod=2000-01&detail=dataonly&format=csvdata')
une=une[['TIME_PERIOD','OBS_VALUE']]
une.TIME_PERIOD=pd.to_datetime(une.TIME_PERIOD, format='%Y-%m-%d')
une.set_index('TIME_PERIOD',inplace=True)
une=une.rename(columns={'OBS_VALUE':'Desempleo'})
une.index=une.index.strftime('%b-%Y')


In [71]:
mro

,MRO
TIME_PERIOD,
Jan-2000,3.00
Feb-2000,3.25
Mar-2000,3.50
Apr-2000,3.75
May-2000,3.75
...,...
Jan-2024,4.50
Feb-2024,4.50
Mar-2024,4.50


In [75]:
data=pd.merge(mro,inf,left_index=True,right_index=True)
data=pd.merge(data,une,left_index=True,right_index=True)
data

,MRO,Inflacion,Desempleo
TIME_PERIOD,,,
Jan-2000,3.00,1.9,9.439001
Feb-2000,3.25,1.9,9.351888
Mar-2000,3.50,2.0,9.261718
Apr-2000,3.75,1.7,9.176502
May-2000,3.75,1.8,9.125067
...,...,...,...
Nov-2023,4.50,2.4,6.537916
Dec-2023,4.50,2.9,6.515967
Jan-2024,4.50,2.8,6.519742
